![](https://la-pistola-records.com/10466-large_default/elton-john-kiki-dee-don%60t-go-breaking-my-heart.jpg)

## "Right from the start..."

My aim with this notebook is to test the researchers' hypothesis that not only can heart failure be accurately predicted using just serum creatinine and ejection fraction, but models using the complete set of variables in their dataset actually perform worse than just the two variable model.

I aim to test this by not only comparing accuracy and F1 scores across a handful of classical and ensemble machine learning models, but also importantly by studying the variation of these two scores for each respective model during cross-validation.  **My central question is does the two-variable model generalize just as well, if not better, and produce more consistent accuracy and F1 scores than the model with all variables.  As a follow-up, if the two-variable model does have more variation, is it acceptable because the scores far exceed the all variable model anyway?**

I then move on to understanding if there is a distinct difference in how accurate the probability scores are for decision predictions for the two-factor model versus the model containing all variables.  Probability scores can be useful measures to guide physicians trust in a model decision but obviously only if they are accurate.

I conclude by creating a Monte Carlo function that tests the significance of each model's results without assuming a distribution type. 

Two notes:
1. My code and descriptions will use the word "focus" to describe datasets and models where only ejection fraction and serum creatinine are used.
2. The model that has all variables actually does not include the variable, "time," which the Kaggle page describes as "follow up period (days)" but the research report does not explain very well.  I excluded this varariable completely from all of my analysis because it seems very likely that this isn't information that a physician would know during the screening where all the other data points are recorded.  Additionally, the "time" variable is extremely predictive and the correlation seems to suggest that a shorter time correlates with higher mortality, which doesn't intuitively make a lot of sense. 

In [ ]:
# Import Libraries

import matplotlib.pyplot as plt
import numpy as np
from numpy.random import Generator, PCG64
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

import os

from scipy.cluster import hierarchy
from scipy.stats import spearmanr

from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [ ]:
# Import Dataset

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

raw_data = pd.read_csv("/kaggle/input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv")

In [ ]:
# Check for missing data and review data types.

print("Missing Data Proportion")
print(pd.isnull(raw_data).mean())

print("Dataset Info")
print(raw_data.info())

In [ ]:
# Set aside a test set.
# Remove the 'time' variable since we cannot know in advance when the next follow-up will be.
# I will only look at the test set when I am done tuning my models and will not make any adjustments to my models after running the test data.

data = raw_data.copy()
data.drop('time', axis=1, inplace=True)
X, y = data.loc[:, data.columns != 'DEATH_EVENT'], data['DEATH_EVENT']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

### Exploratory Data Analysis (EDA)

In [ ]:
training_data = pd.concat([x_train, y_train], axis=1)
eda_scaler = StandardScaler()
scaled_training_data = eda_scaler.fit_transform(training_data)

REFERENCE

* Binary variables: anaemia, diabetes, high_blood_pressure, sex, smoking, DEATH_EVENT
* Continuous variables: age, creatinine_phosphokinase, ejection_fraction, platelets, serum_creatinine, and serum_sodium

Crosstab Analysis

In [ ]:
x_tab_anaemia = pd.crosstab(training_data['anaemia'], training_data['DEATH_EVENT'], normalize='index')
print(x_tab_anaemia)
print( )

x_tab_diabetes = pd.crosstab(training_data['diabetes'], training_data['DEATH_EVENT'], normalize='index')
print(x_tab_diabetes)
print( )

x_tab_hbp = pd.crosstab(training_data['high_blood_pressure'], training_data['DEATH_EVENT'], normalize='index')
print(x_tab_hbp)
# Note: Not having high blood was a decent indicator of survival
print( )

x_tab_sex = pd.crosstab(training_data['sex'], training_data['DEATH_EVENT'], normalize='index')
print(x_tab_sex)
print( )

x_tab_smoking = pd.crosstab(training_data['smoking'], training_data['DEATH_EVENT'], normalize='index')
print(x_tab_smoking)

# High blood pressure is the only stand-out predictor in the training data.
# Given the person has high blood pressure, they have a 38.7% chance of dying.
# Given the person does not have high blood pressure, they have a 28.6% of dying. 

## Quick Plots


The training data includes 164 men (listed as 1) and 90 women (listed as 0).


Note on Sex: 

We know which values in the binary breakdown are associated with male and female by looking at the count of 0s and 1s in the raw data where there are 194 1s and 105 0s. The "Dataset" section of the research reports states, "The patients consisted of 105 women and 194 men, and their ages range between 40 and 95 years old (Table 1)."  The plot below shows us that not only are most of the data male but most of the death events are also male.

In [ ]:
fig_sex = px.histogram(training_data, x='sex', color='DEATH_EVENT')
fig_sex.update_layout(margin=dict(l=20, r=20, t=20, b=20))
fig_sex.show()


Serum Creatinine values are heavily right skewed.  While there are significantly fewer observations at higher levels, the death rate is proportionally much higher.

In [ ]:
fig_serum = px.histogram(training_data, x='serum_creatinine', color='DEATH_EVENT')
fig_serum.update_layout(margin=dict(l=20, r=20, t=20, b=20))
fig_serum.show()

Low levels of ejection fraction appear to correspond with more deaths.

In [ ]:
fig_eject = px.histogram(training_data, x='ejection_fraction', color='DEATH_EVENT')
fig_eject.update_layout(margin=dict(l=20, r=20, t=20, b=20))
fig_eject.show()

As the researchers note in their paper, you can very much imagine a linear boundary separating survivals and deaths in this plot.  The plot shows the importance of low serum_creatinine (most people are between 0.8 - 1.2) and a sort of ideal level of ejection fraction around 35.  Lower ejection fraction than 35 for the same levels of serum creatinine ends up being deadly.

In [ ]:
fig_serum_eject = px.scatter(training_data, x='serum_creatinine', y='ejection_fraction', color='DEATH_EVENT')
fig_serum_eject.update_traces(marker=dict(size=12,
                                          line=dict(width=2,
                                                    color='DarkSlateGrey')),
                              selector=dict(mode='markers'))

fig_serum_eject.update_layout(margin=dict(l=20, r=20, t=20, b=20))
fig_serum_eject.show()

## PCA - Is it worthwhile to use here?

In [ ]:
pca_x = training_data.drop('DEATH_EVENT', axis=1)
pca_scaler = StandardScaler()
pca_x = pca_scaler.fit_transform(pca_x)
pca_list = []
for x in range(1, (pca_x.shape[1]) + 1):
    pca = PCA(n_components=x)
    pca.fit(pca_x)
    pca_list.append(pca.explained_variance_ratio_.sum())

pca_plot = pd.DataFrame({'PCA Components': range(1, (pca_x.shape[1]) + 1),
                         'Explained Variance': pca_list})
pca_fig = px.line(pca_plot, x='PCA Components', y='Explained Variance')
pca_fig.update_layout(margin=dict(l=20, r=20, t=20, b=20))
pca_fig.show()

The smooth curve to this line without any "elbows" makes it difficult to decide which number of PCA components to choose from.  Furthermore, 8 of 11 components are required to achieve 80% explained variance and about 9 are required for 90% explanation -- meaning not a great deal of dimensionality reduction is achieved and PCA is probably not worth using for our models.

## Models

I performed grid searches using all variables (except "time) and just the focus variables.  My goal was to find well parameterized versions of each of the model types, not necessarily to go over the top beating the best performers in the Kaggle circuit.  Again, my goal was to understand key effects to machine learning models from a two-variable and complete variable set approach.

In [ ]:
# Scale the data

scaler = StandardScaler()
x_train_std = scaler.fit_transform(x_train)
x_test_std = scaler.transform(x_test)

One last bit of EDA before jumping in ...

The hierarchical clustering results were fascinating to me.  It's probably important for the two-variable model performance that ejection fraction and serum creatinine do not share a common cluster at all. 

Additionally, if you Google creatinine phosphokinase and diabetes (a clustering pairing), you will find studies linking the two -- one of which can be found [here](https://pubmed.ncbi.nlm.nih.gov/16995840/).

Finally, ejection fraction is ascertained through imaging techniques, while serum sodium is measured through a blood sample.  For areas of the world where imaging a patient is not possible, but blood samples are, it would interesting if machine learning could use serum sodium instead of ejection fraction in a two-variable model to predict heart failure.  

In [ ]:
# Review Hierarchical Clustering & Correlation

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))
corr = spearmanr(x_train_std).correlation
corr_linkage = hierarchy.ward(corr)
dendro = hierarchy.dendrogram(corr_linkage, labels=x_test.columns.tolist(), ax=ax1, leaf_rotation=90)
dendro_idx = np.arange(0, len(dendro['ivl']))

ax2.imshow(corr[dendro['leaves'], :][:, dendro['leaves']])
ax2.set_xticks(dendro_idx)
ax2.set_yticks(dendro_idx)
ax2.set_xticklabels(dendro['ivl'], rotation='vertical')
ax2.set_yticklabels(dendro['ivl'])
fig.tight_layout()
plt.show()

# I used code from the sklearn website for these plots.
# https://scikit-learn.org/stable/auto_examples/inspection/plot_permutation_importance_multicollinear.html#sphx-glr-auto-examples-inspection-plot-permutation-importance-multicollinear-pyThat code is found here.

In [ ]:
# Define grid search function

def grid_search(model, params, x_data=x_train_std, y_data=y_train):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    gcv = GridSearchCV(estimator=model,
                       param_grid=params,
                       scoring=['accuracy', 'f1'],
                       cv=skf,
                       refit='accuracy')
    gcv.fit(x_data, y_data)
    gcv_ser = pd.DataFrame(gcv.cv_results_).loc[gcv.best_index_, :]

    return gcv, gcv_ser


# Create Focus Features DataFrame
focus_features = ['ejection_fraction', 'serum_creatinine']
focus_x_train = x_train.copy()[focus_features]
focus_x_test = x_test.copy()[focus_features]

focus_scaler = StandardScaler()
focus_x_train = focus_scaler.fit_transform(focus_x_train)
focus_x_test = focus_scaler.transform(focus_x_test)

In [ ]:
# Logistic Regression - All features

log_reg = LogisticRegression(random_state=42)
log_param_grid = dict(C=np.arange(0.005, 0.1, 0.001), class_weight=[None, 'Balanced'])
log_param_grid_best = dict(C=[0.088])

log_gcv, log_gcv_ser = grid_search(log_reg, log_param_grid_best)

**Logistic Regression Sub-section - plotting results in 3d**

The 3-D plot shows a feature space defined by the first three PCA components, which themselves explain 38.8% of training data variance.  While it's hard to find a clean boundary of deaths and survivals, there does appear to be a linear hyperplane separating the high probability model predictions from low ones.  

In [ ]:
# Logistic Regression Sub-section - plotting results in 3d

log_preds = log_gcv.predict(x_train_std).tolist()
log_pred_probs = log_gcv.predict_proba(x_train_std).tolist()
winning_probs = [p[i] for p, i in zip(log_pred_probs, log_preds)]
log_pca = PCA(n_components=3)
pca_x = log_pca.fit_transform(pca_x)
exp_var = log_pca.explained_variance_ratio_.sum()
log_df = pd.DataFrame(pca_x, columns=[1, 2, 3])
log_df['Prediction'] = log_preds
log_df['Probability'] = winning_probs
log_df = pd.concat([log_df, training_data['DEATH_EVENT'].reset_index()], axis=1)
log_df['Prediction Check'] = ['correct' if actual == pred else 'incorrect' for
                              actual, pred in zip(log_df['DEATH_EVENT'], log_df['Prediction'])]

log_fig = px.scatter_3d(log_df, x=1, y=2, z=3,
                        color='Probability',
                        symbol='Prediction Check')
log_fig.update_layout(title_text='PCA Axes Explain ' + str(round(exp_var * 100, 1)) + '% of Total Variance',
                      legend=dict(yanchor='top', y=0.99, xanchor='left', x=0.01))
log_fig.show()

In [ ]:
# Logistic Regression - Focus Features

log2_reg = LogisticRegression(random_state=42)
log2_param_grid = dict(C=np.arange(0.1, 1.6, 0.1), class_weight=[None, 'Balanced'])
log2_param_grid_best = dict(C=[0.6])

log2_gcv, log2_gcv_ser = grid_search(log2_reg, log2_param_grid_best, focus_x_train)

In [ ]:
# SVC - All Features

svc = SVC(probability=True, random_state=42)
svc_param_grid = dict(C=np.arange(0.5, 2.1, 0.1), kernel=['rbf', 'linear', 'poly', 'sigmoid'], degree=np.arange(1, 5),
                      class_weight=[None, 'balanced'])

svc_param_grid_best = dict(C=[1.4], class_weight=[None], degree=[1], kernel=['sigmoid'])
svc_gcv, svc_gcv_ser = grid_search(svc, svc_param_grid_best)

In [ ]:
# SVC - Focus Features

svc2 = SVC(probability=True, random_state=42)
svc2_param_grid = dict(C=np.arange(0.05, 0.16, 0.01), kernel=['rbf', 'linear', 'poly', 'sigmoid'],
                       degree=np.arange(1, 5), class_weight=[None, 'balanced'])

svc2_param_grid_best = dict(C=[0.09], class_weight=['balanced'], degree=[1], kernel=['linear'])

svc2_gcv, svc2_gcv_ser = grid_search(svc2, svc2_param_grid_best, focus_x_train)

In [ ]:
# KNN - All Features

knn = KNeighborsClassifier()
knn_param_grid = dict(n_neighbors=np.arange(3, 23, 2), weights=['uniform', 'distance'],
                      algorithm=['auto', 'ball_tree', 'kd_tree', 'brute'], p=[1, 2])

knn_param_grid_best = dict(algorithm=['auto'], n_neighbors=[11], p=[1], weights=['uniform'])

knn_gcv, knn_gcv_ser = grid_search(knn, knn_param_grid_best)


In [ ]:
# KNN - Focus Features

knn2 = KNeighborsClassifier()
knn2_param_grid = dict(n_neighbors=np.arange(3, 23, 2), weights=['uniform', 'distance'],
                       algorithm=['auto', 'ball_tree', 'kd_tree', 'brute'], p=[1, 2])

knn2_param_grid_best = dict(algorithm=['auto'], n_neighbors=[13], p=[1], weights=['distance'])

knn2_gcv, knn2_gcv_ser = grid_search(knn2, knn2_param_grid_best, focus_x_train)

In [ ]:
# Random Forest - All Features

rf = RandomForestClassifier(n_jobs=-1, random_state=921)
rf_param_grid = dict(n_estimators=[10, 15, 25, 50, 75, 100], max_depth=[None, 1, 2, 3, 4, 5],
                     max_features=['auto', 'log2', 'sqrt'], min_samples_leaf=[2, 3, 4, 5],
                     class_weight=[None, 'balanced', 'balanced_subsample'])

rf_param_grid_best = dict(class_weight=['balanced'], max_depth=[3], max_features=['auto'], min_samples_leaf=[3],
                          n_estimators=[15])

rf_gcv, rf_gcv_ser = grid_search(rf, rf_param_grid_best)

In [ ]:
# Random Forest - Focus Features

rf2 = RandomForestClassifier(n_jobs=-1, random_state=921)
rf2_param_grid = dict(n_estimators=[10, 15, 20, 25, 30], max_depth=[None, 5, 6, 7, 10, 15],
                      max_features=['auto', 'log2', 'sqrt'],
                      min_samples_leaf=[2, 3, 4], class_weight=[None, 'balanced', 'balanced_subsample'])

rf2_param_grid_best = dict(class_weight=[None], max_depth=[7], max_features=['auto'], min_samples_leaf=[3],
                           n_estimators=[15])

rf2_gcv, rf2_gcv_ser = grid_search(rf2, rf2_param_grid_best, focus_x_train)

In [ ]:
# Gradient Boosting Classifier - All Features
gbc = GradientBoostingClassifier(random_state=42)

gbc_param_grid = dict(learning_rate=[.05, .06, .08, .09, .1, .25], n_estimators=[50, 75, 100, 125, 150, 200],
                      subsample=[0.5, 0.7, 0.8, 0.9, 1.0], min_samples_leaf=[6, 8, 10, 12, 15],
                      max_depth=[1, 2, 3, 4, 5, 6, 7], max_features=['auto', 'sqrt'], n_iter_no_change=[None, 5])

gbc_param_grid_best = dict(learning_rate=[0.08], max_depth=[4], max_features=['sqrt'], min_samples_leaf=[10],
                           n_estimators=[100], subsample=[1], n_iter_no_change=[None])

gbc_gcv, gbc_gcv_ser = grid_search(gbc, gbc_param_grid_best)

In [ ]:
# Gradient Boosting Classifier - Focus Features

gbc2 = GradientBoostingClassifier(random_state=42)
gbc2_param_grid = dict(learning_rate=np.arange(0.2, 0.8, 0.1), n_estimators=[15, 25, 35, 50, 75],
                       subsample=np.arange(0.2, 1.0, 0.1), min_samples_leaf=np.arange(3, 10, 1),
                       max_depth=[1, 2, 3, 4, 5, 6, 7, 10, 12], max_features=['auto', 'sqrt'],
                       n_iter_no_change=[None, 5])

gbc2_param_grid_best = dict(learning_rate=[0.4], max_depth=[5], max_features=['auto'], min_samples_leaf=[6],
                            n_estimators=[35], subsample=[0.7], n_iter_no_change=[None])

gbc2_gcv, gbc2_gcv_ser = grid_search(gbc, gbc2_param_grid_best, focus_x_train)

In [ ]:
# Build a consolidated results table

res_df = pd.DataFrame(dict(log_all=log_gcv_ser, log_foc=log2_gcv_ser,
                           svc_all=svc_gcv_ser, svc_foc=svc2_gcv_ser,
                           knn_all=knn_gcv_ser, knn_foc=knn2_gcv_ser,
                           rf_all=rf_gcv_ser, rf_foc=rf2_gcv_ser,
                           gbc_all=gbc_gcv_ser, gbc_foc=gbc2_gcv_ser)).T

res_df.head()

In [ ]:
# Create Box Plots for F1 and Accuracy Scores of Models in Cross Validation

f1_res_cols = res_df.columns[(res_df.columns.str.startswith('split')) & (res_df.columns.str.endswith('f1'))]
acc_res_cols = res_df.columns[(res_df.columns.str.startswith('split')) & (res_df.columns.str.endswith('accuracy'))]

This plot is one of the center points of my analysis as it strongly supports the researchers' findings in one plot and makes my hypothesis less plausible.  Recall that my hypothesis was that their two variable model would have higher scoring variation across cross validation splits.  

Each column in the plot represents the F1 scores for 5 randomly selected samples of the training data.  The ideal result would be a very short boxplot located at or near the top of the grid.  Across the models the "foc" variable set (i.e., ejection fraction and serum creatinine variables only) have less variation in the Logistic Regression, KNN, and Random Forest models.  In the cases of the SVC and Gradient Boosted Classifier (GBC) models, the marginally higher variation is eclipsed by both higher median F1 scores and a distribution of scores across cross validation that mostly beat the all variable model.  

In [ ]:
# F1 Box Plots

f1_res = res_df.copy()[f1_res_cols].reset_index()
f1_res_melted = pd.melt(f1_res, id_vars=['index']).drop('variable', axis=1)
f1_res_melted['Model'], f1_res_melted['Variable Set'] = list(zip(*[x.split("_") for x in f1_res_melted['index']]))
fig = px.box(f1_res_melted, x='Model', y='value', color='Variable Set')
fig.update_layout(title_text='F1 Scores', title_x=0.5)
fig.show()

The message is more mixed when we evaluate the same plot for accuracy.  Zooming in on the two best models, Random Forest and GBC, shows the case where the all variable model should be preferred as it has roughly the same variation as the focus variable model, but with a higher median score.  Alternatively, GBC produced the same median scores across variable sets, but with the key difference that the higher variation in the two-model approach meant some cases where results were significantly better.

In [ ]:
# Accuracy Box Plots

acc_res = res_df.copy()[acc_res_cols].reset_index()
acc_res_melted = pd.melt(acc_res, id_vars=['index']).drop('variable', axis=1)
acc_res_melted['Model'], acc_res_melted['Variable Set'] = list(zip(*[x.split("_") for x in acc_res_melted['index']]))
fig = px.box(acc_res_melted, x='Model', y='value', color='Variable Set')
fig.update_layout(title_text='Accuracy Scores', title_x=0.5)
fig.show()

This plot shows the models' probabilities associated with their prediction on the x-axis and the accuracy of their prediction on the y-axis.  The idea here is that if the model tells warns the physician of a death event with a probability score of 80%, we would expect the accuracy of that prediction to be 80% in the long-run.  The gray line in the plot shows this theoretical connection between the x and y axes.  Ideally, the "All" and "Focus" lines would stay very close to the gray line and on occasions where it misses, be above the gray line (indicating a higher accuracy than might have been hoped for given a stated probability). 

The blue and red lines each include all of the machine learning types for their respective set of variables.

While up for debate, this plot tells me that while the F1 boxplot earlier might have championed two-variable models, physicians should be wary of the probability scores such an approach offers them.  One area of particular concern is for probabilities exceeding 80% where the two-variable models do not gain any more accuracy despite increasing their probability scores.  Said another way, if a two-variable model tells you its prediction with a 95% probability, you should assume it's only about 80% accurate.

The All variable model shines a little more brightly in this regard in that it hugs the gray line a little more closely (for the most part).  In the 83% - 89% probability regions its actual accuracy is just as disappointing as the two-variable model (~77%), but for probabilities above that its accuracy actually improves, whereas the two-variable models do not. 

In [ ]:
# Probability Investigation

test_preds_all = []
test_probs_all = []
test_pred_checks_all = []

for mod in [log_gcv, svc_gcv, knn_gcv, rf_gcv, gbc_gcv]:
    preds = mod.predict(x_test_std).tolist()
    probs = mod.predict_proba(x_test_std)
    probs = [p[i] for p, i in zip(probs, preds)]
    pred_checks = [1 if actual == pred else 0 for actual, pred in zip(y_test, preds)]

    test_preds_all = test_preds_all + preds
    test_probs_all = test_probs_all + probs
    test_pred_checks_all = test_pred_checks_all + pred_checks

test_preds_foc = []
test_probs_foc = []
test_pred_checks_foc = []

for mod in [log2_gcv, svc2_gcv, knn2_gcv, rf2_gcv, gbc2_gcv]:
    preds = mod.predict(focus_x_test).tolist()
    probs = mod.predict_proba(focus_x_test)
    probs = [p[i] for p, i in zip(probs, preds)]
    pred_checks = [1 if actual == pred else 0 for actual, pred in zip(y_test, preds)]

    test_preds_foc = test_preds_foc + preds
    test_probs_foc = test_probs_foc + probs
    test_pred_checks_foc = test_pred_checks_foc + pred_checks

mod_list = ['All' for _ in test_preds_all] + ['Focus' for _ in test_preds_foc]

test_prob_df = pd.DataFrame(dict(Prediction=test_preds_all + test_preds_foc,
                                 Probability=test_probs_all + test_probs_foc,
                                 Check=test_pred_checks_all + test_pred_checks_foc,
                                 Variable_Selection=mod_list))

test_prob_df['Binned Probabilities'] = pd.cut(test_prob_df['Probability'], bins=12)
test_prob_grouped = test_prob_df.groupby(['Binned Probabilities', 'Variable_Selection']).mean()['Check'].reset_index()
test_prob_grouped.rename(columns={'Check': 'Accuracy'}, inplace=True)
test_prob_grouped['Expected'] = [x.mid for x in test_prob_grouped['Binned Probabilities']]
test_prob_grouped['Binned Probabilities'] = [str(x) for x in test_prob_grouped['Binned Probabilities']]

fig = px.line(test_prob_grouped, x='Binned Probabilities', y='Accuracy', color='Variable_Selection')
fig.add_trace(go.Scatter(x=test_prob_grouped['Binned Probabilities'],
                         y=test_prob_grouped['Expected'],
                         mode='lines',
                         line=go.scatter.Line(color='gray'),
                         name='Expected',
                         opacity=0.4))
fig.update_traces(mode='markers+lines')
fig.show()

## Statistical Significance - Monte Carlo

In [ ]:
def monte_carlo_sim(y_true, y_preds):
    rg = Generator(PCG64(42))
    mod_acc = accuracy_score(y_true, y_preds)
    mod_f1 = f1_score(y_true, y_preds)
    shuffles = [rg.permutation(y_preds) for _ in range(5000)]
    shuffles_acc = [accuracy_score(y_true, y_shuff) for y_shuff in shuffles]
    shuffles_f1 = [f1_score(y_true, y_shuff) for y_shuff in shuffles]
    perct_acc = np.percentile(shuffles_acc, 95)
    perct_f1 = np.percentile(shuffles_f1, 95)
    return dict(model_accuracy=mod_acc, mc_accuracy=perct_acc, model_f1=mod_f1, mc_f1=perct_f1)


log_mc = monte_carlo_sim(y_test, log_gcv.predict(x_test_std))
svc_mc = monte_carlo_sim(y_test, svc_gcv.predict(x_test_std))
knn_mc = monte_carlo_sim(y_test, knn_gcv.predict(x_test_std))
rf_mc = monte_carlo_sim(y_test, rf_gcv.predict(x_test_std))
gbc_mc = monte_carlo_sim(y_test, gbc_gcv.predict(x_test_std))

log2_mc = monte_carlo_sim(y_test, log2_gcv.predict(focus_x_test))
svc2_mc = monte_carlo_sim(y_test, svc2_gcv.predict(focus_x_test))
knn2_mc = monte_carlo_sim(y_test, knn2_gcv.predict(focus_x_test))
rf2_mc = monte_carlo_sim(y_test, rf2_gcv.predict(focus_x_test))
gbc2_mc = monte_carlo_sim(y_test, gbc2_gcv.predict(focus_x_test))

mc_all_df = pd.DataFrame(columns=['Model', 'Accuracy', 'Accuracy MC 95th', 'F1', 'F1 MC 95th'])
for i, (s, mod) in enumerate([('log_all', log_mc), ('svc_all', svc_mc), ('knn_all', knn_mc),
                              ('rf_all', rf_mc), ('gbc_all', gbc_mc)]):
    mc_all_df.loc[i, :] = [s, mod['model_accuracy'], mod['mc_accuracy'], mod['model_f1'], mod['mc_f1']]

mc_foc_df = pd.DataFrame(columns=['Model', 'Accuracy', 'Accuracy MC 95th', 'F1', 'F1 MC 95th'])
for i, (s, mod) in enumerate([('log_foc', log2_mc), ('svc_foc', svc2_mc), ('knn_foc', knn2_mc),
                              ('rf_foc', rf2_mc), ('gbc_foc', gbc2_mc)]):
    mc_foc_df.loc[i, :] = [s, mod['model_accuracy'], mod['mc_accuracy'], mod['model_f1'], mod['mc_f1']]

This approach to significance testing takes the predictions made by each model, shuffles them using a permuted congruential generator that randomly permutes the values using samples from different distributions and records the accuracy and F1 scores.  This is performed 5,000 times for each model.  The code then finds the 95th percentile accuracy and F1 scores and inserts it into the dataframe.  The main goal here is to avoid inferring a distribution to the dataset while testing the likelihood that a lucky guess from the model could have produced the same accuracy and F1 scores at the 95th percentile level.

The results of this exercise reveal that the logistic regression, random forest, and gradient boosted classifer have statistically significant results in the test sample at the 95th percentile for both accuracy and F1.  The support vector classifier and knn fall short on both accuracy and F1.

The results also show that in the test set then random forest and gradient boosted classifier models achieve accuracy scores of 77.8% and F1 scores of 64.3% and 61.5%, respectively. 

### 95% Statistical Significance Results - All Variable Model

In [ ]:
mc_all_df

There are a few alarming things this next dataframe reveals first and foremost of which is the large drop in accuracy and F1 scores on the test dataset for the two-variable model.  This suggests that while the two-variable model performed well in cross-validation, it did not end up generalizing well to the test data -- possibly the result of a greater suspectibility the two factor model has to overfitting than the model with more variables does.

The support vector classifier ends up being the only statistically significant model in the two-variable set-up.  All other models had results at what the 95th percentile of lucky/random guesses would have achieved. 

### 95% Statistical Significance Results - Two Variable Model

In [ ]:
mc_foc_df

## Conclusion

The study's claim that ejection fraction and serum creatinine alone produced not only adequate but actually better machine learning models to predict heart failure appears to have some important flaws.  First, its ability to correctly guess probability of heart failure stops appropriately progressing past the 80% probability level (an issue the all variable model has less trouble with) and, second, the two-variable models struggles to beat the 95th percentile guesses the Monte Carlo simulation produced in all cases except the SVC model. 

On the training data, the two-variable model shines in cross validation and produces better F1 scores but fails to produce superior accuracy scores.

My best model overall is the all variable random forest model, which produced a 77.8% accuracy score and 64.3% F1 score in testing.  Notwithstanding the results of the Monte Carlo simulation, none of the two variable models produced that level of performance against the test data.

In summary, if we consider the two-variable model's superior F1 results in cross-validation offset by in the inferior ones in accuracy testing and probability performance, then the inability to frequently beat the 95th percentile result of a Monte Carlo simulation and inferior outright results against the test data make the two variable model a worse alternative to the all variable model for physician use.